In [228]:
import numpy as np 
import re
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd

In [229]:
data = pd.read_csv("/home/it/Desktop/Desktop/kauzi/Dataset/spam.csv", encoding='iso-8859-1')

In [230]:
print(data.columns.values)

['v1' 'v2' 'Unnamed: 2' 'Unnamed: 3' 'Unnamed: 4']


In [231]:
y_attribute_sub = dict(zip(['spam', 'ham'], [1, 0]))
for i in range(0, len(data)):
    data['v1'][i] = y_attribute_sub[data['v1'][i]]

In [232]:
print(data.head())

  v1                                                 v2 Unnamed: 2 Unnamed: 3  \
0  0  Go until jurong point, crazy.. Available only ...        NaN        NaN   
1  0                      Ok lar... Joking wif u oni...        NaN        NaN   
2  1  Free entry in 2 a wkly comp to win FA Cup fina...        NaN        NaN   
3  0  U dun say so early hor... U c already then say...        NaN        NaN   
4  0  Nah I don't think he goes to usf, he lives aro...        NaN        NaN   

  Unnamed: 4  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


In [233]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
print(data.head())

  v1                                                 v2
0  0  Go until jurong point, crazy.. Available only ...
1  0                      Ok lar... Joking wif u oni...
2  1  Free entry in 2 a wkly comp to win FA Cup fina...
3  0  U dun say so early hor... U c already then say...
4  0  Nah I don't think he goes to usf, he lives aro...


In [234]:
corpus = list()
for i in range(0, len(data)):
    corpus.extend(re.sub(r'[^\w]', ' ', data['v2'][i]).strip().split())
corpus = list(set(corpus))
print(len(corpus))

10864


In [235]:
encoding = dict(zip(corpus, [i+1 for i in range(0, len(corpus))]))

In [236]:
max_length = 0
for i in range(0, len(data)):
    vector = re.sub(r'[^\w]', ' ', data['v2'][i]).strip().split()
    for j in range(0, len(vector)):
        vector[j] = encoding[vector[j]]
    data['v2'][i] = vector
    if(max_length < len(vector)):
        max_length = len(vector)
print(data.head(), max_length)

  v1                                                 v2
0  0  [4135, 9290, 749, 4714, 8243, 3224, 2064, 4782...
1  0               [6115, 2828, 4496, 1543, 5083, 3635]
2  1  [6532, 6870, 4782, 5804, 5236, 5324, 990, 4859...
3  0  [10408, 9051, 10517, 10407, 7863, 519, 10408, ...
4  0  [1587, 1314, 10443, 7443, 6451, 10753, 59, 485... 190


In [243]:
print(data.head())
X = np.zeros((len(data['v2']), max_length))
for i in range(1, len(data)):
    data['v2'][i].extend([0 for j in range(0, max_length-len(data['v2'][i]))])
    X[i,:] = data['v2'][i]
print(X.shape)

  v1                                                 v2
0  0  [4135, 9290, 749, 4714, 8243, 3224, 2064, 4782...
1  0  [6115, 2828, 4496, 1543, 5083, 3635, 0, 0, 0, ...
2  1  [6532, 6870, 4782, 5804, 5236, 5324, 990, 4859...
3  0  [10408, 9051, 10517, 10407, 7863, 519, 10408, ...
4  0  [1587, 1314, 10443, 7443, 6451, 10753, 59, 485...
(5572, 190)


In [244]:
model = Sequential()
model.add(Dense(100, input_shape=(190, ), activation ='relu', name='dense_7'))
model.add(Dense(30, activation='relu', name='dense_2'))
model.add(Dense(1, activation='sigmoid', name='dense_3'))

In [259]:
model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])

In [260]:
X_train, X_test, y_train, y_test = train_test_split(X, data['v1'], test_size = 0.33, random_state = 42)

In [263]:
print("Shape of input: {} and {} and shape of output: {}".format(X_train.shape, len(X_train[0]), y_train[0]))
model.summary()
model.fit(X_train, y_train, epochs=20, batch_size = 64)

Shape of input: (3733, 190) and 190 and shape of output: 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               19100     
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 22,161
Trainable params: 22,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
3733/3733 [==============================] - 0s 28us/step - loss: 2.1373 - acc: 0.8674
Epoch 2/20
3733/3733 [==============================] - 0s 21us/step - loss: 2.1373 - acc: 0.8674
Epoch 3/20
3733/3733 [==============================] - 0s 24us/step - loss: 2.1373 - acc: 0.8674
Epoch 4/20
3733/3733 [===========================

In [264]:
model.evaluate(X_test, y_test)

1839/1839 [==============================] - 0s 41us/step


[2.208678648742792, 0.8629690049911984]